In [7]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Selenium part

In [25]:
# set options 
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificste-errors')
options.add_argument('--incognito')

# initiate driver
driver = webdriver.Chrome('/Users/CaitlanKrasinski/Desktop/chromedriver', options=options)

# nav to link 
produce = 'https://www.zehrs.ca/food/fruits-vegetables/c/28000?navid=flyout-L2-fruits-vegetables'
driver.get(produce)

# close cookies popup 
time.sleep(8) # wait for page load 

# find button and click button 
cookies_block = driver.find_element_by_xpath(xpath='//*[@id="privacy-policy"]/div/div/button').click()
print('closed cookie blocker')

# wait for page load 
time.sleep(5)

# click load more results button until at bottom 
load_more = driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[6]/div/div[2]/div[4]/div/button')

count = 0
while True:
    try:
        # click the button
        load_more.click()

        # wait for page load 
        time.sleep(5)

        # re-find the button 
        load_more = driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[6]/div/div[2]/div[4]/div/button')
        
        count+=1
        print(f'{count} page(s) loaded')
        
    except:
        break

# double check we reached bottom 
time.sleep(5)
try: 
    load_more = driver.find_element_by_xpath('//*[@id="site-content"]/div/div/div[6]/div/div[2]/div[4]/div/button')
    print('failed run')
except:
    print('reached bottom of page')
    
page_source = driver.page_source

closed cookie blocker
1 page(s) loaded
2 page(s) loaded
3 page(s) loaded
4 page(s) loaded
5 page(s) loaded
6 page(s) loaded
7 page(s) loaded
8 page(s) loaded
9 page(s) loaded
10 page(s) loaded
11 page(s) loaded
12 page(s) loaded
13 page(s) loaded
reached bottom of page


# soup part

In [26]:
soup = BeautifulSoup(page_source, 'lxml')

In [19]:
product_divs = soup.find_all("div", {"class": "product-tracking"})

details = product_divs[0].find_all("div", {"class": "product-tile__details"})

name = details[0].find_all("h3", {"class": "text text--small4 text--left text--default-color product-tile__details__info__name"})[0].text

prod_details = details[0].find_all("div", {"class": "product-tile__details__info__section"})

prod_details2 = prod_details[0].find_all("div", {"class": "product-prices product-prices--product-tile"})

price = prod_details2[0].find_all("span", {"class": "price selling-price-list__item__price selling-price-list__item__price--now-price"})[0].text

per = prod_details2[0].find_all("span", {"class": "price comparison-price-list__item__price"})[0].text


In [20]:
name, price, per

('NeilsonPartly Skimmed Milk 2% MF4 l', '$5.69ea', '$0.14/ 100ml')

### loop through all divs 

In [32]:
product_divs = soup.find_all("div", {"class": "product-tracking"})

df = pd.DataFrame(columns = ['category', 'product_name', 'price', 'per_unit_price'])

for div in product_divs:
    prod_details = div.find_all("div", {"class": "product-tile__details"})
    
    name = prod_details[0].find_all("h3", {"class": "text text--small4 text--left text--default-color product-tile__details__info__name"})[0].text

    prod_details = prod_details[0].find_all("div", {"class": "product-tile__details__info__section"})
    prod_info = prod_details[0].find_all("div", {"class": "product-prices product-prices--product-tile"})
    
    price = prod_info[0].find_all("span", {"class": "price selling-price-list__item__price selling-price-list__item__price--now-price"})[0].text
    
    # some items don't have a per unit price
    try:
        per_unit_price = prod_info[0].find_all("span", {"class": "price comparison-price-list__item__price"})[0].text
    except:
        per_unit_price = None 
        
    df = df.append({'category': 'produce', 
                    'product_name': name, 
                    'price': price,
                    'per_unit_price': per_unit_price}
                   , ignore_index=True)
    

In [24]:
df.to_csv('dairy_data.csv', index=False)